In [1]:
from qcfractal.interface import FractalClient

In [2]:
fc = FractalClient()
fc

FractalClient(server_name='The MolSSI QCArchive Server', address='https://api.qcarchive.molssi.org:443/', username='None')

In [3]:
!openff-benchmark optimize export --help

Usage: openff-benchmark optimize export [OPTIONS]

  Export molecule optimization data from a given dataset into an output
  directory.

  You must provide the dataset name via `-d DATASET_NAME` that you wish to
  export. You must also provide the output directory via `-o
  OUTPUT_DIRECTORY`.

Options:
  -u, --fractal-uri TEXT       Address and port of the QCFractal Server
  -d, --dataset-name TEXT      Dataset name to export molecule optimization
                               data from  [required]

  --delete-existing            Delete existing output directory if it exists
  -p, --processes INTEGER      Number of parellel processes to use for export;
                               if not specified, export done in series.

  -o, --output-directory TEXT  Output directory to use for results  [required]
  --help                       Show this message and exit.


In [7]:
!openff-benchmark optimize export -p 12 -u 'https://api.qcarchive.molssi.org:443' -d 'OpenFF Industry Benchmark Season 1 v1.1' -o data/results-qm

... 'JNS-00396-2' : exported COMPLETE
... 'JNS-00396-3' : exported COMPLETE
... 'JNS-00396-1' : exported COMPLETE
... 'JNS-00211-9' : exported COMPLETE
... 'JNS-00396-6' : exported COMPLETE
... 'JNS-01574-0' : exported COMPLETE
... 'JNS-00396-5' : exported COMPLETE
... 'JNS-00396-4' : exported COMPLETE
... 'JNS-01574-6' : exported COMPLETE
... 'JNS-01574-5' : exported COMPLETE
... 'JNS-01574-1' : exported COMPLETE
... 'JNS-01574-3' : exported COMPLETE
... 'JNS-01574-8' : exported COMPLETE
... 'JNS-01574-4' : exported COMPLETE
... 'JNS-01574-7' : exported COMPLETE
... 'JNS-00296-0' : exported COMPLETE
... 'JNS-00296-1' : exported COMPLETE
... 'JNS-00296-3' : exported COMPLETE
... 'JNS-00956-1' : exported COMPLETE
... 'JNS-01574-2' : exported COMPLETE
... 'JNS-00296-6' : exported COMPLETE
... 'JNS-00956-0' : exported COMPLETE
... 'JNS-00296-7' : exported COMPLETE
... 'JNS-00296-5' : exported COMPLETE
... 'JNS-00956-3' : exported COMPLETE
... 'JNS-00956-2' : exported COMPLETE
... 'JNS-009

## Create submittable MM dataset from endpoints of QM dataset

In [1]:
!openff-benchmark optimize create-submittable -o "dataset-mm.json.bz2" --dataset-name "OpenFF Industry Benchmark Season 1 - MM v1.1" --season "1:2" data/results-qm/default/

[13:38:53] Warning: molecule is tagged as 3D, but all Z coords are zero
[13:38:53] Warning: molecule is tagged as 3D, but all Z coords are zero
[13:38:53] Warning: molecule is tagged as 3D, but all Z coords are zero
[13:38:53] Warning: molecule is tagged as 3D, but all Z coords are zero
[13:38:54] Warning: molecule is tagged as 3D, but all Z coords are zero
[13:38:54] Warning: molecule is tagged as 3D, but all Z coords are zero
[13:38:54] Warning: molecule is tagged as 3D, but all Z coords are zero
[13:38:54] Warning: molecule is tagged as 3D, but all Z coords are zero
[13:38:54] Warning: molecule is tagged as 3D, but all Z coords are zero
[13:38:54] Warning: molecule is tagged as 3D, but all Z coords are zero
[13:38:54] Warning: molecule is tagged as 3D, but all Z coords are zero
[13:39:02] Warning: molecule is tagged as 3D, but all Z coords are zero
[13:39:02] Warning: molecule is tagged as 3D, but all Z coords are zero
[13:39:02] Warning: molecule is tagged as 3D, but all Z coords a

## Stats on dataset

In [1]:
import json
from openff.qcsubmit.datasets import OptimizationDataset
from openff.qcsubmit.serializers import deserialize

In [2]:
ds = deserialize('dataset-mm.json.bz2')

In [5]:
optds = OptimizationDataset.parse_obj(ds)

/home/david/.conda/envs/openff-benchmark-optimization-dev/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C37H50N7O4 has a net charge of 1.0.
  UserWarning,
/home/david/.conda/envs/openff-benchmark-optimization-dev/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C15H25N2O has a net charge of 1.0.
  UserWarning,
/home/david/.conda/envs/openff-benchmark-optimization-dev/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C24H31N2O4 has a net charge of 1.0.
  UserWarning,
/home/david/.conda/envs/openff-benchmark-optimization-dev/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C29H40NO6 has a net charge of 1.0.
  UserWarning,
/home/david/.conda/envs/openff-benchmark-optimization-dev/lib/python3.7/site-packages/openff/qcsubmit/validators.py:253: UserWarning: The molecule C18ClH17NO4S has a net charge of -1.0.
  UserWarning,
/home/dav

In [8]:
full_dataset = optds

In [9]:
from rdkit.Chem import Descriptors
import numpy as np

In [11]:
confs = np.array([len(mol.conformers) for mol in full_dataset.molecules])
elements = set()
for entry in full_dataset.dataset.values():
    elements.update(entry.initial_molecules[0].symbols)

print("Unique list of elements          ", elements)
print("Number of unique molecules       ", full_dataset.n_molecules)
print("Number of optimizations         ", full_dataset.n_records)
print("Number of conformers min mean max",
      confs.min(), "{:6.2f}".format(confs.mean()), confs.max())

masses = []
for molecule in full_dataset.molecules:
    rd_mol = molecule.to_rdkit()
    mass = Descriptors.ExactMolWt(rd_mol)
    masses.append(mass)

print(f'Mean molecular weight: {np.mean(np.array(masses)):.2f}')
print(f'Max molecular weight: {np.max(np.array(masses)):.2f}')
print("Charges:", sorted(set(m.total_charge/m.total_charge.unit for m in full_dataset.molecules)))


Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Bonds with undefined stereochemistry are:
 - Bond 12 (atoms 12-13 of element (C-C)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Bonds with undefined stereochemistry are:
 - Bond 12 (atoms 12-13 of element (C-C)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Bonds with undefined stereochemistry are:
 - Bond 12 (atoms 12-13 of element (C-C)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Bonds with undefined stereochemistry are:
 - Bond 12 (atoms 12-13 of element (C-C)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Bonds with und

Unique list of elements           {'N', 'F', 'Cl', 'O', 'Br', 'S', 'H', 'C', 'P'}


RDKit WARNING: [18:06:55] Conflicting single bond directions around double bond at index 27.
RDKit WARNING: [18:06:55]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [18:06:55] Conflicting single bond directions around double bond at index 27.
RDKit WARNING: [18:06:55]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [18:06:55] Conflicting single bond directions around double bond at index 27.
RDKit WARNING: [18:06:55]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [18:06:55] Conflicting single bond directions around double bond at index 27.
RDKit WARNING: [18:06:55]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [18:06:55] Conflicting single bond directions around double bond at index 27.
RDKit WARNING: [18:06:55]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [18:06:55] Conflicting single bond 

Number of unique molecules        9467
Number of optimizations          71655
Number of conformers min mean max 1   1.00 1


RDKit WARNING: [18:14:13] Conflicting single bond directions around double bond at index 27.
RDKit WARNING: [18:14:13]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [18:14:13] Conflicting single bond directions around double bond at index 27.
RDKit WARNING: [18:14:13]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [18:14:13] Conflicting single bond directions around double bond at index 27.
RDKit WARNING: [18:14:13]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [18:14:13] Conflicting single bond directions around double bond at index 27.
RDKit WARNING: [18:14:13]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [18:14:13] Conflicting single bond directions around double bond at index 27.
RDKit WARNING: [18:14:13]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [18:14:13] Conflicting single bond 

Mean molecular weight: 358.69
Max molecular weight: 1104.40


Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Bonds with undefined stereochemistry are:
 - Bond 12 (atoms 12-13 of element (C-C)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Bonds with undefined stereochemistry are:
 - Bond 12 (atoms 12-13 of element (C-C)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Bonds with undefined stereochemistry are:
 - Bond 12 (atoms 12-13 of element (C-C)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Bonds with undefined stereochemistry are:
 - Bond 12 (atoms 12-13 of element (C-C)

Warning (not error because allow_undefined_stereo=True): Unable to make OFFMol from RDMol: RDMol has unspecified stereochemistry. Bonds with und

Charges: [-2.0, -1.0, 0.0, 1.0, 2.0]


In [27]:
{key: value.dict() for key, value in optds.qc_specifications.items()}

{'smirnoff99Frosst-1.1.0': {'method': 'smirnoff99frosst-1.1.0',
  'basis': 'smirnoff',
  'program': 'openmm',
  'spec_name': 'smirnoff99Frosst-1.1.0',
  'spec_description': 'default smirnoff99Frosst-1.1.0 optimization spec',
  'store_wavefunction': 'none',
  'implicit_solvent': None},
 'openff-1.0.0': {'method': 'openff-1.0.0',
  'basis': 'smirnoff',
  'program': 'openmm',
  'spec_name': 'openff-1.0.0',
  'spec_description': 'default openff-1.0.0 optimization spec',
  'store_wavefunction': 'none',
  'implicit_solvent': None},
 'openff-1.1.1': {'method': 'openff-1.1.1',
  'basis': 'smirnoff',
  'program': 'openmm',
  'spec_name': 'openff-1.1.1',
  'spec_description': 'default openff-1.1.1 optimization spec',
  'store_wavefunction': 'none',
  'implicit_solvent': None},
 'openff-1.2.1': {'method': 'openff-1.2.1',
  'basis': 'smirnoff',
  'program': 'openmm',
  'spec_name': 'openff-1.2.1',
  'spec_description': 'default openff-1.2.1 optimization spec',
  'store_wavefunction': 'none',
  'im

In [28]:
optds.scf_properties

[<SCFProperties.Dipole: 'dipole'>,
 <SCFProperties.Quadrupole: 'quadrupole'>,
 <SCFProperties.WibergLowdinIndices: 'wiberg_lowdin_indices'>,
 <SCFProperties.MayerIndices: 'mayer_indices'>]

In [16]:
optds.molecules_to_file("dataset.smi", "smi")

## Metadata edits

In [24]:
import openforcefield
import openff.benchmark
import rdkit

In [22]:
openff.benchmark.__version__

'2021.04.09.0+27.ga4304fe.dirty'

In [19]:
optds.metadata.dataset_name = optds.dataset_name
optds.metadata.dataset_name

'OpenFF Industry Benchmark Season 1 - MM v1.1'

In [32]:
# change the metadata
full_dataset.metadata.submitter = "dotsdl"
full_dataset.metadata.long_description_url = ("https://github.com/openforcefield/qca-dataset-submission/tree/master/"
                                                "submissions/"
                                                "2021-07-28-OpenFF-Industry-Benchmark-Season-1-MM-v1.1")
full_dataset.metadata.elements = elements
full_dataset.provenance = {"openff-qcsubmit": openff.qcsubmit.__version__,
                            "openff-toolkit": openforcefield.__version__,
                            "rdkit": rdkit.__version__,
                            "openff-benchmark": "2021.04.09.0+27.ga4304fe.dirty"}

full_dataset.metadata.long_description = 'MM combination of all publicly chosen compound sets by industry partners from the OpenFF season 1 industry benchmark.'
full_dataset.metadata.short_description = "The public molecules from the OpenFF Industry Benchmark, MM optimizations following QM."

In [33]:
optds.metadata.dict()

{'submitter': 'dotsdl',
 'creation_date': datetime.date(2021, 7, 22),
 'collection_type': 'OptimizationDataset',
 'dataset_name': 'OpenFF Industry Benchmark Season 1 - MM v1.1',
 'short_description': 'The public molecules from the OpenFF Industry Benchmark, MM optimizations following QM.',
 'long_description_url': HttpUrl('https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2021-07-28-OpenFF-Industry-Benchmark-Season-1-MM-v1.1', scheme='https', host='github.com', tld='com', host_type='domain', path='/openforcefield/qca-dataset-submission/tree/master/submissions/2021-07-28-OpenFF-Industry-Benchmark-Season-1-MM-v1.1'),
 'long_description': 'MM combination of all publicly chosen compound sets by industry partners from the OpenFF season 1 industry benchmark.',
 'elements': {'Br', 'C', 'Cl', 'F', 'H', 'N', 'O', 'P', 'S'}}

In [34]:
optds.export_dataset("dataset.json.bz2")